In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import optuna
import math
import pandas as pd
from tqdm import tqdm
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder

class BetaEncoder(object):
        
    def __init__(self, group):
        self.group = group
        self.stats = None
        
    # get counts from df
    def fit(self, df, target_col):
        self.prior_mean = np.mean(df[target_col]) 
        stats = df[[target_col, self.group]].groupby(self.group)
        stats = stats.agg(['sum', 'count'])[target_col]    
        stats.rename(columns={'sum': 'n', 'count': 'N'}, inplace=True)
        stats.reset_index(level=0, inplace=True)           
        self.stats = stats

    # extract posterior statistics
    def transform(self, df, stat_type, N_min=1):
        
        df_stats = pd.merge(df[[self.group]], self.stats, how='left')
        n = df_stats['n'].copy()
        N = df_stats['N'].copy()
        
        # fill in missing
        nan_indexs = np.isnan(n)
        n[nan_indexs] = self.prior_mean
        N[nan_indexs] = 1.0
        
        # prior parameters
        N_prior = np.maximum(N_min-N, 0)
        alpha_prior = self.prior_mean*N_prior
        beta_prior  = (1-self.prior_mean)*N_prior
        
        # posterior parameters
        alpha =  alpha_prior + n
        beta =  beta_prior  + N-n
        
        # calculate statistics
        if stat_type=='mean':
            num = alpha
            dem = alpha+beta
                    
        elif stat_type=='mode':
            num = alpha-1
            dem = alpha+beta-2
            
        elif stat_type=='median':
            num = alpha-1/3
            dem = alpha+beta-2/3
        
        elif stat_type=='var':
            num = alpha*beta
            dem = (alpha+beta)**2*(alpha+beta+1)
                    
        elif stat_type=='skewness':
            num = 2*(beta-alpha)*np.sqrt(alpha+beta+1)
            dem = (alpha+beta+2)*np.sqrt(alpha*beta)

        elif stat_type=='kurtosis':
            num = 6*(alpha-beta)**2*(alpha+beta+1) - alpha*beta*(alpha+beta+2)
            dem = alpha*beta*(alpha+beta+2)*(alpha+beta+3)
            
        # replace missing
        value = num/dem
        value[np.isnan(value)] = np.nanmedian(value)
        return value

In [3]:
raw_data = pd.read_csv("./data/training_data.csv")
# display(raw_data)

train = pd.read_csv("./data/train_feat.csv")
# display(train.info())
feat_cols = ['土地面積','移轉層次','總樓層數','屋齡','建物面積','車位面積','車位個數','橫坐標','縱坐標','主建物面積','陽台面積',
                '附屬建物面積','N_lib_2000','avg_distances_高中','avg_distances_國小','avg_distances_火車','avg_distances_醫療',
                'avg_distances_公車','avg_distances_國中','avg_distances_大學','avg_distances_便利','avg_distances_AT',
                'avg_distances_金融','avg_distances_捷運','avg_distances_郵局',
                'avg_tax','density','edu_p', '縣市_台北市','縣市_台中市','縣市_台南市','縣市_新北市','縣市_高雄市','縣市_桃園市']
cat_cols = ['使用分區','主要用途','主要建材','建物型態','縣市_鄉鎮市區']

raw_data['縣市_鄉鎮市區'] = raw_data['縣市'] + '_' + raw_data['鄉鎮市區']

selected_X = train[feat_cols]
cat_X = raw_data[cat_cols+['單價']]
X = pd.concat([selected_X, cat_X], axis=1)
Y = pd.read_csv("./data/train_output.csv")
Y = np.log(Y)

test = pd.read_csv("./data/test_feat.csv")
raw_test_data = pd.read_csv("./data/public_dataset.csv")
raw_test_data['縣市_鄉鎮市區'] = raw_test_data['縣市'] + '_' + raw_test_data['鄉鎮市區']
selected_X = test[feat_cols]
cat_X = raw_test_data[cat_cols]
X_test = pd.concat([selected_X, cat_X], axis=1)


# display(X.head())
# display(Y)
# print(X_test.info())
# print(X_test['主要用途'].value_counts())
# print(X_test['使用分區'].value_counts())
# print(X_test['主要建材'].value_counts())
# print(X_test['建物型態'].value_counts())
# print(X_test['縣市_鄉鎮市區'].value_counts())

In [ ]:
## best_p_list = []
best_s_list = []
best_score = math.inf
# N_min_list = [10,20,30,50,70,110]
# for i in N_min_list:
def objective(trial):
    global best_score
    params = {
        "metric": "mape",
        "num_iterations": trial.suggest_int("num_iterations", 300, 1000), 
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 64, 2**10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "max_bin": trial.suggest_int("max_bin", 128, 1024),
    }
    N_min = 72

    for col in cat_cols:
        # print(f"now at {col}")
        le = LabelEncoder()
        le.fit(np.concatenate([X[col], X_test[col]]))
        X[col] = le.transform(X[col])
        X_test[col] = le.transform(X_test[col])

    x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=507)
    x_train.reset_index(inplace=True, drop=True)
    x_valid.reset_index(inplace=True, drop=True)
    y_train.reset_index(inplace=True, drop=True)
    y_valid.reset_index(inplace=True, drop=True)
    y_valid = np.exp(y_valid)
    
    
    for c in cat_cols:
        # fit encoder
        be = BetaEncoder(c)
        be.fit(x_train, '單價')
        # mean
        feature_name = f'{c}_mean'
        x_train[feature_name] = be.transform(x_train, 'mean', N_min)
        x_valid[feature_name]  = be.transform(x_valid,  'mean', N_min)
        X_test[feature_name]  = be.transform(X_test,  'mean', N_min)

    x_train = x_train.drop(['單價']+cat_cols,axis=1)
    x_valid = x_valid.drop(['單價']+cat_cols,axis=1)
    X_test_d = X_test.drop(cat_cols,axis=1)

    
    model = LGBMRegressor(**params)
    model.fit(x_train, y_train)
    pred = model.predict(x_valid)
    pred = np.exp(pred)
    mape = mean_absolute_percentage_error(y_valid, pred)
    
    if(mape < best_score):
        x_train.to_csv('data/x_train.csv', index=False)
        y_train.to_csv('data/y_train.csv', index=False)
        x_valid.to_csv('data/x_valid.csv', index=False)
        y_valid.to_csv('data/y_valid.csv', index=False)
        X_test_d.to_csv('data/x_test.csv', index=False)
        # public_pred = model.predict(X_test_d)
        # public_pred.to_csv('pred.csv', index=False)
        best_score = mape

    return mape



In [5]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=80)
print('Best hyperparameters:', study.best_params)
print('Best MAPE:', study.best_value)

[I 2023-11-13 05:25:40,625] A new study created in memory with name: no-name-88dbd9c8-b714-41a7-9331-adc591691b51
C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=47, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=47
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 05:25:42,961] Trial 0 finished with value: 0.09346875073169705 and parameters: {'num_iterations': 310, 'learning_rate': 0.036514289392378226, 'num_leaves': 791, 'subsample': 0.5922799226763265, 'colsample_bytree': 0.7777033506802487, 'min_data_in_leaf': 47, 'max_bin': 592, 'N_min': 287}. Best is trial 0 with value: 0.09346875073169705.
C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 05:25:55,913] Trial 1 finished with value: 0.09343368612031824 and parameters: {'num_iterations': 701, 'learning_rate': 0.025889860366320672, 'num_leaves': 443, 'subsample': 0.9487872028660727, 'colsample_bytree': 0.9888038202083119, 'min_data_in_leaf': 10, 'max_bin': 170, 'N_min': 141}. Best is trial 1 with value: 0.09343368612031824.
C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 05:33:11,280] Trial 78 finished with value: 0.09094144694860364 and parameters: {'num_iterations': 644, 'learning_rate': 0.013905978895550139, 'num_leaves': 597, 'subsample': 0.6985080504691474, 'colsample_bytree': 0.6053403775604315, 'min_data_in_leaf': 19, 'max_bin': 781, 'N_min': 249}. Best is trial 47 with value: 0.09056746318625584.
C:\Users\kogby\miniconda3\envs\Jupyter\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-11-13 05:33:26,533] Trial 79 finished with value: 0.0908112474662479 and parameters: {'num_iterations': 558, 'learning_rate': 0.027154244871209758, 'num_leaves': 570, 'subsample': 0.6515887604858498, 'colsample_bytree': 0.6700470924398497, 'min_data_in_leaf': 10, 'max_bin': 1001, 'N_min': 269}. Best is trial 47 with value: 0.09056746318625584.


Best hyperparameters: {'num_iterations': 531, 'learning_rate': 0.022211533935262557, 'num_leaves': 517, 'subsample': 0.5781755393245138, 'colsample_bytree': 0.6449946295668693, 'min_data_in_leaf': 14, 'max_bin': 445, 'N_min': 72}
Best MAPE: 0.09056746318625584


In [10]:
X_TRAIN = pd.read_csv('data/x_train.csv')
Y_TRAIN = pd.read_csv('data/y_valid.csv')
display(Y_TRAIN.isna().value_counts())

單價   
False    2351
Name: count, dtype: int64